In [ ]:
#@title Run CVaR Widget (Live Market Data/Graphs, All S&P 500 Tickers) { display-mode: "form" }
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import pytz
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import seaborn as sns


# --- 1. Get S&P 500 tickers and major ETFs ---
try:
    snp = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
    sp500_tickers = snp['Symbol'].tolist()
except Exception:
    # fallback: minimal set
    sp500_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA']

major_etfs = ['SPY', 'VOO', 'QQQ', 'VTI']
all_tickers = sorted(list(set(sp500_tickers + major_etfs)))

# --- 2. Ticker autofill dropdown ---
ticker_input = widgets.Combobox(
    value='SPY',
    placeholder='Type or select a ticker (SPY, AAPL, MSFT, GOOG, META, TSLA, etc.)',
    options=all_tickers,
    description='Type or select a ticker (SPY, AAPL, MSFT, GOOG, META, TSLA, etc.)',
    ensure_option=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# --- 3. Expiry and Strike Dropdowns ---
def get_available_expiries(ticker):
    try:
        ticker_obj = yf.Ticker(ticker)
        today = datetime.datetime.today()
        available_expiries = [
            e for e in ticker_obj.options
            if 5 <= (datetime.datetime.strptime(e, "%Y-%m-%d") - today).days <= 365
        ]
        return available_expiries
    except Exception as ex:
        return []

expiry_dropdown = widgets.Dropdown(
    options=get_available_expiries('SPY'),
    description='Choose Expiration Date:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

strike_dropdown = widgets.Dropdown(
    options=[],
    description='Choose Strike Price:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

button = widgets.Button(description="Run CVaR Analysis", button_style='primary')
output = widgets.Output()

# Update strike dropdown when expiry changes or ticker changes
def update_strike_dropdown(*args):
    ticker = ticker_input.value.upper()
    expiry = expiry_dropdown.value
    if expiry:
        try:
            ticker_obj = yf.Ticker(ticker)
            opt_chain = ticker_obj.option_chain(expiry)
            calls = opt_chain.calls.dropna(subset=['strike', 'bid', 'ask'])
            calls['mid'] = (calls['bid'] + calls['ask']) / 2
            calls = calls.sort_values('strike')
            current_price = round(ticker_obj.history(period="1d")['Close'].iloc[-1], 2)
            atm_strike = min(calls['strike'], key=lambda x: abs(x - current_price))
            available_strikes = sorted(calls[(calls['strike'] >= atm_strike - 100) & (calls['strike'] <= atm_strike + 100)]['strike'].unique())
            strike_dropdown.options = available_strikes
            closest_strike = min(available_strikes, key=lambda x: abs(x - current_price))
            strike_dropdown.value = closest_strike
        except Exception as ex:
            strike_dropdown.options = []
            with output:
                clear_output()
                display(HTML(f"<div style='color:red'><strong>❌ Error:</strong> Couldn't load strikes for {ticker}: {str(ex)}</div>"))

# Update expiry dropdown when ticker changes
def update_expiry_dropdown(*args):
    ticker = ticker_input.value.upper()
    available_expiries = get_available_expiries(ticker)
    expiry_dropdown.options = available_expiries
    if available_expiries:
        expiry_dropdown.value = available_expiries[0]
    else:
        strike_dropdown.options = []
        with output:
            clear_output()
            display(HTML(f"<div style='color:red'><strong>❌ Error:</strong> No expiries available for {ticker}.</div>"))

ticker_input.observe(update_expiry_dropdown, names='value')
expiry_dropdown.observe(update_strike_dropdown, names='value')
update_expiry_dropdown()
update_strike_dropdown()

# Callback for CVaR button
def on_button_click(b):
    with output:
        clear_output(wait=True)
        try:
            ticker = ticker_input.value.upper()
            expiry = expiry_dropdown.value
            target_strike = strike_dropdown.value

            ticker_obj = yf.Ticker(ticker)
            current_price = round(ticker_obj.history(period="1d")['Close'].iloc[-1], 2)
            opt_chain = ticker_obj.option_chain(expiry)
            calls = opt_chain.calls.copy().dropna()
            calls['mid'] = (calls['bid'] + calls['ask']) / 2
           ## calls["mid"] = 0 ## SIMULATE CLOSED MARKET market close
            calls = calls.sort_values('strike').reset_index(drop=True)
            atm_strike = min(calls['strike'], key=lambda x: abs(x - current_price))
            calls = calls[(calls['strike'] >= atm_strike - 100) & (calls['strike'] <= atm_strike + 100)]

            target_row = calls[calls['strike'] == target_strike]

            if target_row.empty:
                call_mid = np.nan
                f_k = np.nan
                VaR = current_price - target_strike
                CVaR = np.nan
                error_message = f"❌ Target strike {target_strike} not found. Choose another strike or Date."
            else:
                live_quote = target_row.iloc[0]
                call_mid = target_row['mid'].values[0]
                f_k = np.nan
                VaR = current_price - target_strike
                CVaR = np.nan
                error_message = None

            # Live quote panel
            if not target_row.empty:
                last_price_display = f"{live_quote['lastPrice']:.2f}" if 'lastPrice' in live_quote and pd.notna(live_quote['lastPrice']) else "N/A"
                change_display = f"{live_quote['change']:.2f}" if 'change' in live_quote and pd.notna(live_quote['change']) else "N/A"
                percent_change_display = f"{live_quote['percentChange']:.2f}%" if 'percentChange' in live_quote and pd.notna(live_quote['percentChange']) else "N/A"
                volume_display = f"{int(live_quote['volume'])}" if 'volume' in live_quote and pd.notna(live_quote['volume']) else "N/A"
                open_interest_display = f"{int(live_quote['openInterest'])}" if 'openInterest' in live_quote and pd.notna(live_quote['openInterest']) else "N/A"
                implied_volatility_display = f"{live_quote['impliedVolatility']:.4f}" if 'impliedVolatility' in live_quote and pd.notna(live_quote['impliedVolatility']) else "N/A"
                in_the_money_display = 'Yes' if 'inTheMoney' in live_quote and live_quote['inTheMoney'] else ('No' if 'inTheMoney' in live_quote and not live_quote['inTheMoney'] else "N/A")
                currency_display = live_quote['currency'] if 'currency' in live_quote and pd.notna(live_quote['currency']) else "N/A"

                display(HTML(f"""
                    <div style='background-color:#000000; color:white; padding:15px; border-radius:10px; font-family:monospace'>
                        <strong style="font-size:16px;">Live Market Quote for {ticker.upper()} Call Option (Strike = {target_strike}, Expiry = {expiry})</strong><br><br>
                        <table style='width:100%; border-collapse:collapse; text-align:center; font-size:14px;'>
                            <thead>
                                <tr style='color:white; border-bottom:1px solid white;'>
                                    <th>Strike</th>
                                    <th>Last</th>
                                    <th>Bid</th>
                                    <th>Ask</th>
                                    <th>Change</th>
                                    <th>% Change</th>
                                    <th>Volume</th>
                                    <th>Open Interest</th>
                                    <th>ImpliedVol</th>
                                    <th>In Money</th>
                                    <th>Currency</th>
                                </tr>
                            </thead>
                            <tbody>
                                <tr style='color:white;'>
                                    <td>{live_quote['strike']}</td>
                                    <td>{last_price_display}</td>
                                    <td>{live_quote['bid']:.2f}</td>
                                    <td>{live_quote['ask']:.2f}</td>
                                    <td>{change_display}</td>
                                    <td>{percent_change_display}</td>
                                    <td>{volume_display}</td>
                                    <td>{open_interest_display}</td>
                                    <td>{implied_volatility_display}</td>
                                    <td>{in_the_money_display}</td>
                                    <td>{currency_display}</td>
                                </tr>
                            </tbody>
                        </table>
                    </div>
                """))
            elif error_message:
                display(HTML(f"""
                    <div style='background-color:#ffcccc;color:black;padding:12px;border-radius:8px'>
                    <strong>❌ Error:</strong> {error_message}
                    </div>
                """))

            # Estimate dc/dK and avg_dc_dk
            calls['mid_smooth'] = calls['mid'].interpolate().rolling(window=5, center=True).mean()
            calls['dc_dk'] = calls['mid_smooth'].diff() / calls['strike'].diff()
            calls['avg_dc_dk'] = calls['dc_dk'].rolling(window=2, center=True).mean()

            # Risk-free rate from IRX
            irx = yf.Ticker("^IRX")
            rf_raw = irx.history(period="1d")['Close'].iloc[-1]
            if pd.isna(rf_raw):
                raise ValueError("⚠️ Could not fetch risk-free rate. Try during market hours.")
            r = round(rf_raw / 100, 4)

            # Time to expiration
            pacific = pytz.timezone('US/Pacific')
            now = datetime.datetime.now(pacific)
            expiry_date = pacific.localize(datetime.datetime.strptime(expiry, '%Y-%m-%d'))
            t = (expiry_date - now).days / 365
            discount_factor = np.exp(r * t)
            expiry_readable = expiry_date.strftime('%B %d, %Y')

            # Compute f_k
            calls['f_k'] = 1 + calls['avg_dc_dk'] * discount_factor
            calls['f_k'] = calls['f_k'].clip(lower=0, upper=1)
            calls['1_minus_f_k'] = 1 - calls['f_k']

            target = calls[calls['strike'] == target_strike].dropna()
            if target.empty:
                raise ValueError(f"❌ Target strike {target_strike} not found. Choose another strike or Date.")
            f_k = target['f_k'].values[0]
            call_mid = target['mid'].values[0]

            VaR = current_price - target_strike
            CVaR = VaR + (discount_factor * call_mid) / (1 - f_k)

            closed_market = call_mid == 0 or f_k == 1 or pd.isna(call_mid) or pd.isna(f_k)

            if closed_market:
                display(HTML(f"""
                    <div style='background-color:#ffe0e0; color:black; padding:15px; border-radius:10px; font-weight:bold'>
                    ⚠️⚠️⚠️ WARNING: Live Option data IS NOT available during non-trading hours<br>
                    Re-run this code during regular market hours (6:30 AM – 1:00 PM PST / 9:30 AM – 4:00 PM EST) ⚠️⚠️⚠️
                    </div>
                """))

            display(HTML(f"""
                <div style='background-color:#001f3f; color:white; padding:15px; border-radius:10px; font-family:monospace'>
                    <strong style="font-size:16px;">VaR / CVaR Analysis for {ticker.upper()} (Strike = {target_strike}):</strong><br><br>
                  🔹 Current {ticker.upper()} Price: ${current_price}<br>
                  🔹Risk-Free Rate (3M): {r:.2%}<br>
                  🔹 Time to Expiry: {t:.3f} years<br>
                  🔹  Date: {now.strftime('%Y-%m-%d %I:%M %p %Z')}<br><br>
                  🔸  Midpoint Call Price: ${call_mid:.2f}<br>
                  🔸 P({ticker.upper()} > {target_strike}) at expiry: {(1 - f_k):.4f}<br>
                  🔸 VaR: ${VaR:.2f}<br>
                  🔸  CVaR: ${CVaR:.2f}<br>
                  🔸  P({ticker.upper()} ≤ {target_strike}): {f_k:.4f} <br><br>


                  <strong style="font-size:16px;">⚠️ NOTE: If options data looks off the charts, it's due to option pricing illiqudity. It's better to select a middle strike or a more traded stock.</strong>
                </div>
            """))
            calls = calls.dropna(subset=['f_k'])
            calls = calls[(calls['f_k'] >= 0) & (calls['f_k'] <= 1)]


            # convert ticker to company name
            company_name = ticker_obj.info.get('longName') or ticker_obj.info.get('shortName') or ticker.upper()

   # --- Plot CVaR Visualization ---
            fig, ax = plt.subplots(figsize=(10, 6))
            fig.patch.set_facecolor('#0f172a')
            ax.set_facecolor('#0f172a')
            sns.lineplot(data=calls, x='strike', y='f_k', color='#00FFFF', linewidth=2, label='CDF: P(≤ K)', ax=ax)
            ax.axvline(x=target_strike, color='red', linestyle='--', linewidth=2, label=f'VaR threshold (K = {target_strike})')
            tail_strikes = calls[calls['strike'] >= target_strike]
            ax.fill_between(tail_strikes['strike'], tail_strikes['f_k'], 1, color='green', alpha=0.3, label='Probability-Weighted Exposure to Gains')
            ax.axhline(y=f_k, color='orange', linestyle='--', linewidth=1.5, label=f'P(≤ K) = {f_k:.2%}')
            plt.suptitle(f"Conditional Value at Risk Visualization for {company_name} Strike {target_strike} at Expiration {expiry_readable}", fontsize=16, weight='bold', color='white')
            plt.title(f"{f_k * 100:.2f}% risk neutral probability {company_name} below {target_strike} (assuming VaR = Strike)", fontsize=12, color='lightgray')
            ax.set_xlabel("Strike Price (K)", fontsize=13, color='white')
            ax.set_ylabel("P(≤ K)", fontsize=13, color='white')
            ax.tick_params(colors='white')
            ax.set_xlim(calls['strike'].min(), calls['strike'].max())
            ax.set_ylim(0, 1.05)
            ax.legend(loc='lower right', fontsize=11, facecolor='#1e293b', edgecolor='white', labelcolor='white')
            sns.despine()
            plt.tight_layout()
            ax.grid(True, which='both', linestyle=':', linewidth=0.7, alpha=0.7)
            if closed_market:
                ax.text(
                    0.5, 0.5,
                    "⚠️ Option Data Unavailable\nTry again during market hours ⚠️",
                    fontsize=16, weight='bold', color='orange',
                    ha='center', va='center', transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.6, boxstyle='round,pad=0.5')
                )
            plt.show()

            # --- New CVaR Payoff Plot ---
            payoff = np.maximum(calls['strike'] - target_strike, 0)
            conditional = calls[calls['strike'] > target_strike].copy()
            conditional['payoff'] = conditional['strike'] - target_strike
            cvar_value = conditional['payoff'].mean()

            fig, ax = plt.subplots(figsize=(10, 6))
            fig.patch.set_facecolor('#0f172a')
            ax.set_facecolor('#0f172a')
            plt.suptitle(f"Conditional Value at Risk Average Payoff for {company_name} Strike {target_strike}, at Expiration {expiry_readable}", fontsize=16, weight='bold', color='white')
            plt.title(f"${cvar_value:.2f} average payoff if  {company_name} ends above all possible outcomes of price > {target_strike}", fontsize=12, color='lightgray')
            ax.plot(calls['strike'], payoff, color='cyan', linewidth=2, label='Payoff: max(Stock - K, 0)')
            ax.axvline(x=target_strike, color='red', linestyle='--', linewidth=2, label=f'VaR threshold (K = {target_strike})')
            ax.fill_between(conditional['strike'], conditional['payoff'], color='green', alpha=0.3, label='Tail region (CVaR area)')
            ax.axhline(y=cvar_value, color='orange', linestyle='--', linewidth=2, label=f'CVaR ≈ ${cvar_value:.2f}')
            ax.text(
                target_strike + 10, cvar_value + 5,
                f"CVaR = ${cvar_value:.2f}",
                color='orange', fontsize=12, weight='bold'
            )
            ax.set_xlabel("Strike Price (K)", fontsize=13, color='white')
            ax.set_ylabel("Payoff", fontsize=13, color='white')
            ax.tick_params(colors='white')
            ax.set_xlim(calls['strike'].min(), calls['strike'].max())
            ax.legend(loc='upper left', fontsize=11, facecolor='#1e293b', edgecolor='white', labelcolor='white')
            ax.grid(True, linestyle=':', linewidth=0.7, alpha=0.7)
            sns.despine()
            plt.tight_layout()
            if closed_market:
                ax.text(
                    0.5, 0.5,
                    "⚠️ Option Data Unavailable\nTry again during market hours ⚠️",
                    fontsize=16, weight='bold', color='orange',
                    ha='center', va='center', transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.6, boxstyle='round,pad=0.5')
                )
            plt.show()

# --- Plot CDF and mark VaR threshold ---
            alpha = 0.95  # 5% tail
            calls_sorted = calls.sort_values('strike').dropna(subset=['f_k'])

            # Find strike where f(K) ≈ alpha (VaR threshold)
            var_row = calls_sorted.iloc[(calls_sorted['f_k'] - alpha).abs().argsort()[:1]]
            var_strike = float(var_row['strike'].values[0])
            var_prob = float(var_row['f_k'].values[0])
            VaR_dollars = current_price - var_strike

            # Calculate CVaR (Expected payoff in tail)
            tail_region = calls_sorted[calls_sorted['strike'] > var_strike].copy()
            tail_region = tail_region.astype({'strike': float, 'f_k': float})
            tail_region['delta_f'] = tail_region['f_k'].diff().fillna(0).abs()
            tail_region['payoff'] = tail_region['strike'] - var_strike
            numerator = (tail_region['payoff'] * tail_region['delta_f']).sum()
            denominator = (1 - alpha)
            cvar_value = numerator / denominator if denominator > 0 else np.nan


            UNUSUAL_CVAR = 0.5 * current_price
            cvar_warning = ""    # Always define before
            if cvar_value == 0 or cvar_value < 0 or cvar_value > UNUSUAL_CVAR:
                cvar_warning = (f"Unusual CVaR due to illiquidity in option prices\n"
                                "Try selecting a closer expiration date for this chart or more liquid stock.")


         # (display warning after creating plot) ^ above is setting the warning


           # Plotting
            fig, ax = plt.subplots(figsize=(10, 6))
            fig.patch.set_facecolor('#0f172a')
            ax.set_facecolor('#0f172a')

            plt.suptitle(f"VaR Estimation from Risk-Neutral CDF (5% upper tail) for expiration {expiry_readable} of {company_name}", fontsize=18, weight='bold', color='white')
            plt.title(
            f"If {company_name} reaches the upper {(1 - alpha) * 100:.0f}% tail of outcomes "
            f"based on the current market price of \\${current_price:.2f}, "
            f"average payoff is \\${cvar_value:.2f}",
            fontsize=12,
            color='lightgray'
              )

            sns.lineplot(data=calls_sorted, x='strike', y='f_k', color='cyan', label='Risk-Neutral CDF: f(K)', linewidth=2)
            ax.axhline(y=alpha, color='green', linewidth=1.5, label=f"Tail α = 0.05")
            ax.axvline(x=var_strike, color='pink', linestyle='--', linewidth=2, label=f"VaR Strike = {var_strike:.1f}")
            ax.scatter([var_strike], [var_prob], color='red', s=70, edgecolor='white', zorder=5)

            # CVaR label only
            ax.text(
                var_strike + 10, alpha + 0.03,
                f"CVaR = ${cvar_value:.2f}",
                color='lime', fontsize=12, weight='bold'
            )

            # Axes and layout
            ax.set_xlabel("Strike Price (K)", fontsize=13, color='white')
            ax.set_ylabel("P(SPY ≤ K)", fontsize=13, color='white')
            ax.tick_params(colors='white')
            ax.set_xlim(calls_sorted['strike'].min(), calls_sorted['strike'].max())
            ax.set_ylim(0, 1.05)
            ax.axhline(
            y=0, color='lime', linestyle='-', linewidth=1.5,
            label=f"CVaR ≈ ${cvar_value:.2f}", visible=False
                )
            ax.legend(loc='upper left', fontsize=11, facecolor='#1e293b', edgecolor='white', labelcolor='white')
            sns.despine()
            plt.tight_layout()
            if closed_market:
                ax.text(
                    0.5, 0.5,
                    "⚠️ Option Data Unavailable\nTry again during market hours ⚠️",
                    fontsize=16, weight='bold', color='orange',
                    ha='center', va='center', transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.6, boxstyle='round,pad=0.5')
                )
            if cvar_warning:
                ax.text(
                    0.5, 0.5,
                    cvar_warning,
                    fontsize=13, weight='bold', color='pink',
                    ha='center', va='center', transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.6, boxstyle='round,pad=0.5')
    )


            plt.show()

            # --- Plot 1 - f(K) ---

            fig, ax = plt.subplots(figsize=(10, 6))
            fig.patch.set_facecolor('#0f172a')
            ax.set_facecolor('#0f172a')

            sns.lineplot(
                data=calls, x='strike', y='1_minus_f_k',
                color='#009900', linewidth=2,
                label=f'1 - f(K): P({company_name} > K)', ax=ax
            )
            ax.axvline(
                x=target_strike, color='#FF00FF',
                linestyle='--', linewidth=1.5,
                label=f'Target Strike = {target_strike}'
            )
            ax.scatter(
                [target_strike], [1 - f_k],
                color='#009900', edgecolor='white',
                s=70, zorder=5,
                label=f'Approx Probability = {(1 - f_k):.2%}'
            )
            plt.suptitle(
                f"Risk-Neutral Probability {company_name} Ends Above {target_strike} at Expiration {expiry_readable} (1 Minus Risk Neutral CDF)",
                fontsize=16, weight='bold', color='white'
            )
            plt.title(
                f"{(1 - f_k) * 100:.2f}% {company_name} above {target_strike}; "
                f"{(f_k) * 100:.2f}% below {target_strike}",
                fontsize=12, color='lightgray'
            )
            ax.set_xlabel("Strike Price (K)", fontsize=13, color='white')
            ax.set_ylabel(f"P({company_name} > K)", fontsize=13, color='white')
            ax.tick_params(colors='white')
            ax.set_xlim(calls['strike'].min(), calls['strike'].max())
            ax.set_ylim(0, 1)
            ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
            ax.legend(loc='upper right', fontsize=12, facecolor='#1e293b', edgecolor='white', labelcolor='white')
            sns.despine()
            plt.tight_layout()

            if closed_market:
                ax.text(
                    0.5, 0.5,
                    "⚠️ Option Data Unavailable\nTry again during market hours ⚠️",
                    fontsize=16, weight='bold', color='orange',
                    ha='center', va='center', transform=ax.transAxes,
                    bbox=dict(facecolor='black', alpha=0.6, boxstyle='round,pad=0.5')
                )

            plt.show()


        except Exception as e:
            display(HTML(f"""
                <div style='background-color:#ffcccc;color:black;padding:12px;border-radius:8px'>
                <strong>❌ Error:</strong> {str(e)}
                </div>
            """))

# Step 6: Display widgets
button.on_click(on_button_click)
display(HTML(
    "<b>Click Play Button (next to show code) to enable buttons or to refresh glitches</b><br><br>"
    "<b>Type a company ticker, select expiry and strike, then click Run CVaR Analysis</b>"
))
display(ticker_input, expiry_dropdown, strike_dropdown, button, output)


Combobox(value='SPY', description='Type or select a ticker (SPY, AAPL, MSFT, GOOG, META, TSLA, etc.)', ensure_…

Dropdown(description='Choose Expiration Date:', layout=Layout(width='50%'), options=('2025-06-17', '2025-06-18…

Dropdown(description='Choose Strike Price:', index=38, layout=Layout(width='50%'), options=(np.float64(520.0),…

Button(button_style='primary', description='Run CVaR Analysis', style=ButtonStyle())

Output()

# Explanation

**What is CVaR?**

CVaR is expected shortfall and is a risk assessment measure. CVaR is derived by taking a weighted average of the losses in the tail of the distribution of possible returns, beyond the value at risk (VaR) cutoff point. It answers the question “If a loss exceeds the VaR threshold, what is the average loss in that worst-case tail?”

My tool uses CVaR upside analysis and answers the question, “If a gain exceeds the VaR threshold (e.g., SPY ends above a selected strike), what is the average gain conditional on being in that upper set of outcomes?”


**What is VaR?**

VaR or value at risk is the given threshold for CVaR. It answers “what is the worse-case loss I can expect with __% confidence over a given time?”

In this tool, VaR combined with CVaR, answers the question, “What is the average gain I can expect if SPY lands in the top 5% of possible outcomes (upper tail)?”

In the first two graphs, we manually set the strike = VaR. Now in the third graph, we derive the strike corresponding to a 5% tail (α = 0.95) based on the risk-neutral distribution implied by market option prices. This gives us the strike that the market believes has only a 5% chance of being exceeded, making it the natural VaR threshold. The CVaR is then computed by taking the average payoff across all strike prices beyond that VaR point to find the expected average payoff if SPY reached the top 5% tail outcomes.


**Where do the probabilities come from?**

The probabilities come from differentiating option prices with respect to strike which allows us to extract the implied risk neutral distrubtion of SPY at expiration. Options reflect forward looking market expectations of what the market thinks SPY will be worth at expiration under a risk neutral measure. This is the risk neutral distribution embedded in option prices.

The risk neutral distribution is embedded in options due to no-arb pricing theory that assumes all assets grow at risk free rate when discounting under risk neutral measure. From there, you can back out cumulative probabilites from observed option prices. In practice, you’d use real world measure; however risk neutral measure is a good proxy for shorter time frames. The difference between risk neutral and real world diminishes as time approaches 0 (shorter time frames). Under real world probabilities, CVaR is lower than under risk neutral probabilities.

The methodology comes from Giovanni Barone Adesi papers which uses European put option to extract the implied risk neutral distribution of asset prices at expiration. I take this methodology and use American call options to extract these distributions. Most models require assumptions about the distribution of returns, but here we directly extract this distribution based on forward looking expectations of the market embedded in option pricing.


**Notes**
When CVaR gives average expected payoff, it is not profit. The profit would be expected payoff minus premium for option.
If the graphs have jagged lines or the estimates are off the charts, it means either noisy, illiquid, or stale option prices

Hows it different from delta?
Delta gives a pointwise slope with respect to price, while f(K) gives the full risk-neutral probability density over strikes, which is essential to integrate and compute expected losses or gains.This allows you to look across multiple outcomes, weighted by their likelihood.



